# Project Information

The project is based on Classification problem. For that purpose we have used tree based machine learning models such as Decision Trees and enesemble methods such as Random Forest Classifier. We further Perform hyperparameter tunning for both models to find best parameters for training our models to avoid any overfitting and underfitting problems

In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv #downloading the dataset

--2024-05-04 16:40:39--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: ‘housing.csv’

housing.csv         100%[===================>]   1.36M  --.-KB/s    in 0.06s   

2024-05-04 16:40:40 (24.5 MB/s) - ‘housing.csv’ saved [1423529/1423529]



# Importing libraries

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

# Preparing Dataset

In [3]:
df = pd.read_csv('/content/housing.csv')
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [5]:
df.shape

(20640, 10)

In [6]:
df.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [7]:
median = df['total_bedrooms'].median()
df['total_bedrooms'].fillna(median,inplace=True)

In [8]:
df.isnull().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

In [9]:
df['ocean_proximity'].value_counts()

ocean_proximity
<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: count, dtype: int64

In [11]:
df_class = df.copy()
mean = df_class['median_house_value'].mean()
df_class['above_average'] = np.where(df_class['median_house_value']>=mean,1,0)

In [13]:
df_class=df_class.drop('median_house_value',axis=1)

In [14]:
df_class.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,above_average
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,NEAR BAY,1
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,NEAR BAY,1
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,NEAR BAY,1
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,NEAR BAY,1
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,NEAR BAY,1


# Setting up the validation Framework

In [15]:
df_full_train,df_test = train_test_split(df_class,test_size=0.2,random_state = 1)
df_train,df_val = train_test_split(df_full_train,test_size=0.25,random_state = 1)

df_full_train = df_full_train.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.above_average.values
y_val = df_val.above_average.values
y_test = df_test.above_average.values

del df_train['above_average']
del df_val['above_average']
del df_test['above_average']


In [18]:
train_dicts = df_train.to_dict(orient='records')
val_dicts = df_val.to_dict(orient='records')
test_dicts = df_test.to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X_train=dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)
X_test = dv.transform(test_dicts)

In [21]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(12384, 13)
(4128, 13)
(4128, 13)


# Decision Tree Classifier & Tunning

Decision trees are powerful tools in the field of machine learning and data analysis. They are a versatile and interpretable way to make decisions and predictions based on a set of input features. Imagine a tree-like structure where each internal node represents a feature or attribute, each branch signifies a decision or outcome, and each leaf node provides a final prediction or classification.

Decision trees are widely used for tasks such as classification and regression. They are known for their simplicity and ease of interpretation, making them a valuable resource for understanding and solving complex problems. In this blog post, we’ll delve into the world of decision trees, exploring how they work, and how to build them.

In [27]:
dt=DecisionTreeClassifier()
dt.fit(X_train,y_train)
y_train_pred = dt.predict(X_train)  #training accuracy
y_val_pred = dt.predict(X_val)      #validation accuray
y_test_pred = dt.predict(X_test)    #testing accurracy

print('Training accuracy is: ',roc_auc_score(y_train,y_train_pred))
print('Validation accuracy is: ',roc_auc_score(y_val,y_val_pred))
print('Testing accuracy is: ',roc_auc_score(y_test,y_test_pred))


Training accuracy is:  1.0
Validation accuracy is:  0.8322140271197193
Testing accuracy is:  0.8395432692307692


**Overfitting Problem**

This is called overfitting. Overfitting is when our model simply memorizes the data, but it memorizes in such a way that when it sees a new example it doesn’t know what to do with this example. So it memorizes the training data but it fails to generalize. The reason why this happens to decision trees is that the model creates a specific rule for each example. That works fine for training data, but it doesn’t work for any unseen example.

**Parameter Tunning**

Parameter tuning involves selecting the best parameters for training. In this context, ‘tuning’ means choosing parameters in a way that maximizes or minimizes a chosen performance metric (such as AUC or RMSE) on the validation and testing. In the case of AUC, our goal is to maximize it on the validation  and testing set by finding the parameter values that yield the highest score.

Some of the key parameters include:

**criterion**: This parameter determines the impurity measure used for splitting. You can choose between ‘gini’ for Gini impurity and ‘entropy’ for information gain. The choice of criterion can significantly impact the quality of the splits in the decision tree.

**max_depth**: This parameter controls the maximum depth of the decision tree. It plays a crucial role in preventing overfitting by limiting the complexity of the tree. Selecting an appropriate value for max_depth helps strike a balance between model simplicity and complexity.

**min_samples_leaf**: This parameter specifies the minimum number of samples required in a leaf node. It influences the granularity of the splits. Smaller values can result in finer splits and a more complex tree, while larger values lead to coarser splits and a simpler tree.

In [33]:
scores = []

for d in [3,4,5,6,7,10,15,20, None]:
  for s in [1,2,5,10,15,20,100,200,500]:
    dt=DecisionTreeClassifier(max_depth=d,min_samples_leaf=s)
    dt.fit(X_train,y_train)

    y_val_pred = dt.predict(X_val)
    y_test_pred = dt.predict(X_test)
    auc_val=roc_auc_score(y_val,y_val_pred)
    auc_test=roc_auc_score(y_test,y_test_pred)
    scores.append((d,s,auc_val,auc_test))

columns = ['max_depth', 'min_samples_leaf', 'auc_val','auc_test']
df_scores = pd.DataFrame(scores, columns=columns)

df_scores.sort_values(by='auc_val', ascending=False).head()


,max_depth,min_samples_leaf,auc_val,auc_test
58,15.0,15,0.861491,0.851339
76,NaN,15,0.861324,0.851068
67,20.0,15,0.861324,0.851068
68,20.0,20,0.853106,0.857395
77,NaN,20,0.853106,0.857596


so we will train the model using parameters max_depth = 15 and min_samples_leaf=0.86

In [40]:
dt=DecisionTreeClassifier(max_depth=15,min_samples_leaf=15,random_state=42)
dt.fit(X_train,y_train)
y_train_pred = dt.predict(X_train)  #training accuracy
y_val_pred = dt.predict(X_val)      #validation accuray
y_test_pred = dt.predict(X_test)    #testing accurracy

print('Training accuracy is: ',roc_auc_score(y_train,y_train_pred))
print('Validation accuracy is: ',roc_auc_score(y_val,y_val_pred))
print('Testing accuracy is: ',roc_auc_score(y_test,y_test_pred))


Training accuracy is:  0.8963397044551433
Validation accuracy is:  0.8614905634278482
Testing accuracy is:  0.8515389328808446


# Random Forest Classifier & Tunning

This method of aggregating multiple models is applicable to any type of model. However, in this case, we specifically focus on using decision trees. When we ensemble decision trees, we create what is known as a ‘random forest.’

But you might wonder, why do we call it a ‘random forest’ and not just a ‘forest’? The reason is that if we take the same application (with the same set of features) and build the same group of trees with identical parameters, the resulting trees would also be identical. These identical trees would produce exactly the same probability of default, and therefore, the average would be the same as well. Essentially, we would be training the same model five times, which is not what we want.

To avoid this redundancy and bring the ‘random’ element into play, a ‘random forest’ introduces variability during the training process by using bootstrapped samples and random subsets of features, leading to a more diverse and robust ensemble of decision trees.

In a random forest, each model gets a random subset of features, and the response generated by the ensemble will be a probability of default. This is the fundamental concept behind a random forest.

In [46]:
rf=RandomForestClassifier(n_estimators=10,random_state=42)
rf.fit(X_train,y_train)
y_train_pred = rf.predict(X_train)  #training accuracy
y_val_pred = rf.predict(X_val)      #validation accuray
y_test_pred = rf.predict(X_test)    #testing accurracy

print('Training accuracy is: ',roc_auc_score(y_train,y_train_pred))
print('Validation accuracy is: ',roc_auc_score(y_val,y_val_pred))
print('Testing accuracy is: ',roc_auc_score(y_test,y_test_pred))


Training accuracy is:  0.9909419757080258
Validation accuracy is:  0.861256643718073
Testing accuracy is:  0.8520927601809953


Again overfitting problem. Lets done with a Hyperparameter Tunning

In [47]:
scores = []
for d in [5,10,15,20]:
  for s in [1,3,5,10,50,200]:
    for n in range(10,201,10):
      rf = RandomForestClassifier(n_estimators=n,max_depth=d,min_samples_leaf=s,random_state=1)
      rf.fit(X_train,y_train)
      y_pred=rf.predict(X_val)
      auc=roc_auc_score(y_val,y_pred)
      scores.append((d,s,n,auc))
columns = ['max_depth','min_samples_lef','n_estimators','auc']
df_scores = pd.DataFrame(scores,columns=columns)
df_scores.sort_values(by='auc', ascending=False).head()

,max_depth,min_samples_lef,n_estimators,auc
377,20,1,180,0.877937
258,15,1,190,0.877855
378,20,1,190,0.877855
259,15,1,200,0.877772
379,20,1,200,0.877564


In [49]:
rf=RandomForestClassifier(n_estimators=190,max_depth=15,min_samples_leaf=1,random_state=1)
rf.fit(X_train,y_train)
y_train_pred = rf.predict(X_train)  #training accuracy
y_val_pred = rf.predict(X_val)      #validation accuray
y_test_pred = rf.predict(X_test)    #testing accurracy

print('Training accuracy is: ',roc_auc_score(y_train,y_train_pred))
print('Validation accuracy is: ',roc_auc_score(y_val,y_val_pred))
print('Testing accuracy is: ',roc_auc_score(y_test,y_test_pred))

Training accuracy is:  0.9952879373965273
Validation accuracy is:  0.8778545627741284
Testing accuracy is:  0.8702630090497737
